In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
import requests
import zipfile
import io
from datetime import datetime
import pyquery
import thread

## Environment

In [23]:
from dwre_tools.env import get_default_project, get_project, load_config, get_environment

In [24]:
PROJECT = get_project("partner")
ENV = get_environment("sandbox", PROJECT)

## Login Session

In [61]:
webdavsession = requests.session()
webdavsession.auth=(ENV["username"], ENV["password"],)
bmsession = requests.session()

In [66]:
bmsession.post("https://{}/on/demandware.store/Sites-Site/default/ViewApplication-ProcessLogin".format(ENV["server"]),
               data=dict(
                    LoginForm_Login=ENV["username"],
                    LoginForm_Password=ENV["password"],
                    LocaleID="",
                    LoginForm_RegistrationDomain="Sites",
                    login=""
                ), verify=False)

/Users/clavery/.venv/default/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/Users/clavery/.venv/default/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


<Response [200]>

## Utilities

In [122]:
def wait_for_import():
    response = bmsession.get("https://{}/on/demandware.store/Sites-Site/default/ViewSiteImpex-Status".format(ENV["server"]))
    response_q = pyquery.PyQuery( response.content)
    log_link = response_q.find("a:contains('Site Import')").eq(0).attr("href")
    finished = False
    while not finished: 
        log_response = bmsession.get(log_link)
        if "finished successfully" in log_response.content:
            finished = True
        else:
            thread.sleep(2)

## Bootstrap

In [48]:
TOOL_VERSION = "1"
BOOTSTRAP_META = """<?xml version="1.0" encoding="UTF-8"?>
<metadata xmlns="http://www.demandware.com/xml/impex/metadata/2006-10-31">
    <type-extension type-id="OrganizationPreferences">
        <custom-attribute-definitions>
            <attribute-definition attribute-id="dwreMigrateCurrentVersion">
                <display-name xml:lang="x-default">DWRE Migrate Current Version</display-name>
                <type>string</type>
                <mandatory-flag>false</mandatory-flag>
                <externally-managed-flag>false</externally-managed-flag>
                <min-length>0</min-length>
            </attribute-definition>
            <attribute-definition attribute-id="dwreMigrateToolVersion">
                <display-name xml:lang="x-default">DWRE Migrate Tool Version</display-name>
                <type>string</type>
                <mandatory-flag>false</mandatory-flag>
                <externally-managed-flag>false</externally-managed-flag>
                <min-length>0</min-length>
                <default-value>{0}</default-value>
            </attribute-definition>
        </custom-attribute-definitions>
        <group-definitions>
            <attribute-group group-id="dwreMigrate">
                <display-name xml:lang="x-default">DWREMigrate</display-name>
                <attribute attribute-id="dwreMigrateCurrentVersion"/>
                <attribute attribute-id="dwreMigrateToolVersion"/>
            </attribute-group>
        </group-definitions>
    </type-extension>
</metadata>
""".format(TOOL_VERSION)

In [49]:
PREFERENCES = """<?xml version="1.0" encoding="UTF-8"?>
<preferences xmlns="http://www.demandware.com/xml/impex/preferences/2007-03-31">
    <custom-preferences>
        <all-instances>
            <preference preference-id="dwreMigrateToolVersion">1</preference>
        </all-instances>
    </custom-preferences>
</preferences>
"""

VERSION = """###########################################
# Generated file, do not edit.
# Copyright (c) 2015 by Demandware, Inc.
###########################################
15.5.2
"""

In [107]:
dest_file = "DWREMigrateBootstrap_{}".format(datetime.now().isoformat())

In [108]:
bootstrap_package_file = io.BytesIO()
bootstrap_package_zip = zipfile.ZipFile(bootstrap_package_file, "w")

In [109]:
bootstrap_package_zip.writestr("{}/version.txt".format(dest_file), VERSION)
bootstrap_package_zip.writestr("{}/preferences.xml".format(dest_file), PREFERENCES)
bootstrap_package_zip.writestr("{}/meta/system-objecttype-extensions.xml".format(dest_file), BOOTSTRAP_META)
bootstrap_package_zip.close()

In [110]:
with open("test.zip", "wb") as f:
    bootstrap_package_file.seek(0)
    f.write(bootstrap_package_file.read())

### Upload Bootstrap

In [111]:
dest_url = "https://{0}/on/demandware.servlet/webdav/Sites/Impex/src/instance/{1}.zip".format(
    ENV["server"], dest_file)
webdavsession.put(dest_url, data=bootstrap_package_file, verify=False)

/Users/clavery/.venv/default/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


<Response [201]>

### Activate Bootstrap

In [121]:
response = bmsession.post("https://{}/on/demandware.store/Sites-Site/default/ViewSiteImpex-Dispatch".format(ENV["server"]),
                          data={"import" :"", "ImportFileName" : dest_file + ".zip", "realmUse": "False"})

/Users/clavery/.venv/default/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [123]:
wait_for_import()

/Users/clavery/.venv/default/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/Users/clavery/.venv/default/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


## Load Migrations Context

## Determine Required Migrations

## Perform Migrations